# Import

In [ ]:
import os
import torch
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)


# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import numpy as np
import warnings
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from collections import Counter
from collections.abc import Iterable
import argparse
import os
import sys
import time
import pickle
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
# import torch_geometric
from sklearn.metrics import r2_score

from torch.nn import Linear

# from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, BatchNorm
# from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
# from torch_geometric.data import DataLoader as PYG_DataLoader
# from torch_geometric.data import Data
from itertools import cycle

drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
edge_path = "/content/drive/MyDrive/miRNA/graph/edge.pkl"
edge_attr_path = "/content/drive/MyDrive/miRNA/graph/edge_attr.pkl"
node_path = "/content/drive/MyDrive/miRNA/graph/node.pkl"
mesh_emb_path = '/content/drive/MyDrive/miRNA/embedding/mesh_embs.npy'
mesh_vocab_path = '/content/drive/MyDrive/miRNA/embedding/mesh_vocab.npy'

In [ ]:
all_x_disease_path = '/content/drive/MyDrive/miRNA/last_data/cir_X_Disease_v3.npy'
all_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/cir_X_RNA_v3.npy'
all_pre_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/cir_pre_RNA_x_v3.npy'
all_y_soft_path = '/content/drive/MyDrive/miRNA/last_data/cir_soft_y_v3.npy'
all_y_hard_path = '/content/drive/MyDrive/miRNA/last_data/cir_hard_y_v3.npy'
cir_hair_x_path = '/content/drive/MyDrive/miRNA/last_data/cir_pre_hair_X_v3.npy'
snd_x_path = "/content/drive/MyDrive/miRNA/last_data/cir_snd_X_v3.npy"

In [ ]:
all_orig_x_disease = np.load(all_x_disease_path, allow_pickle=True)
all_orig_x_rna = np.load(all_x_rna_path, allow_pickle=True)
all_orig_pre_x_rna = np.load(all_pre_x_rna_path, allow_pickle=True)

all_hair_x = np.load(cir_hair_x_path, allow_pickle=True)
snd_x = np.load(snd_x_path, allow_pickle=True)

all_orig_Y_soft = np.load(all_y_soft_path, allow_pickle=True)
all_orig_Y_hard = np.load(all_y_hard_path, allow_pickle=True)

In [ ]:
with open(edge_path, 'rb') as f:
    edge_d = pickle.load(f)

with open(edge_attr_path, 'rb') as f:
    edge_attr_d = pickle.load(f)

with open(node_path, 'rb') as f:
    node_d = pickle.load(f)

In [ ]:
# for regression
x_disease=[]
x_rna=[]
hair_x=[]
y_soft=[]
# y_hard=[]
max_len = 0
for i in range(len(all_orig_x_disease)):
  value = edge_d.get(all_orig_x_disease[i])
  if value != None:
    x_disease.append(all_orig_x_disease[i])
    # x_rna.append(all_orig_pre_x_rna[i]+all_orig_x_rna[i])
    x_rna.append(all_orig_x_rna[i])
    if max_len < len(all_orig_x_rna[i]):max_len = len(all_orig_x_rna[i])
    hair_x.append([*all_hair_x[i], *snd_x[i]])
    y_soft.append(all_orig_Y_soft[i])
    # # y_hard.append(all_orig_Y_hard[i])
    # 'down', 'ns', 'up'
    # print(i)

x_disease=np.array(x_disease)
x_rna=np.array(x_rna)
hair_x=np.array(hair_x)
y_soft=np.array(y_soft)
# # y_hard=np.array(y_hard)

In [ ]:
max_len

27

In [ ]:
print(len(all_orig_x_disease))
print(len(all_orig_x_rna))
print(len(all_orig_Y_soft))
print(len(all_orig_Y_hard))
print(len(all_orig_pre_x_rna))
print(len(all_hair_x))
print(len(snd_x))

7835
7835
7835
7835
7835
7835
7835


In [ ]:
print(len(x_disease))
print(len(x_rna))
print(len(y_soft))
# print(len(y_hard))
print(len(hair_x))

6587
6587
6587
6587


In [ ]:
keys = np.load(r'/content/drive/MyDrive/miRNA/embedding/vocab_npa.npy', allow_pickle=True)
values = np.load(r'/content/drive/MyDrive/miRNA/embedding/embs_npa.npy', allow_pickle=True)
word2vec = dict(zip(keys, values))

# Dataset

In [ ]:
def make_dict(mer = {}, n = 1):
  pointer = 4
  if mer == {}:
    for i in range(5):
      mer[str(i)] = str(pointer)
      pointer += 1
  else:
    old_dict = mer
    new_dict = {}
    for key in old_dict:
      for i in range(5):
        new_key = key + str(i)
        if key[0] != '0' and key[-1] != '0':
          new_dict[new_key] = str(pointer)
          pointer += 1
        if key[-1] == '0' and i == 0:
          new_dict[new_key] = str(pointer)
          pointer += 1
    mer = new_dict
  # print(mer)
  if n != 0:
    return make_dict(mer, n-1)
  else:
    return mer



In [ ]:
mer_dict=make_dict(n=1)

new_mer_dict ={}
for i, key in enumerate(mer_dict):
  new_mer_dict[key] = int(mer_dict[key])
new_mer_dict 

{'00': 4,
 '10': 5,
 '11': 6,
 '12': 7,
 '13': 8,
 '14': 9,
 '20': 10,
 '21': 11,
 '22': 12,
 '23': 13,
 '24': 14,
 '30': 15,
 '31': 16,
 '32': 17,
 '33': 18,
 '34': 19,
 '40': 20,
 '41': 21,
 '42': 22,
 '43': 23,
 '44': 24}

In [ ]:
mer_dict=make_dict(n=0)


new_mer_dict ={}
for i, key in enumerate(mer_dict):
  new_mer_dict[key] = int(mer_dict[key])

a_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
twoMer_dict = {**a_dict, **new_mer_dict}
twoMer_dict

{'00': 4,
 '10': 5,
 '11': 6,
 '12': 7,
 '13': 8,
 '14': 9,
 '20': 10,
 '21': 11,
 '22': 12,
 '23': 13,
 '24': 14,
 '30': 15,
 '31': 16,
 '32': 17,
 '33': 18,
 '34': 19,
 '40': 20,
 '41': 21,
 '42': 22,
 '43': 23,
 '44': 24,
 '[CLS]': 1,
 '[MASK]': 3,
 '[PAD]': 0,
 '[SEP]': 2}

In [ ]:
mer_dict=make_dict(n=1)


new_mer_dict ={}
for i, key in enumerate(mer_dict):
  new_mer_dict[key] = int(mer_dict[key])

a_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
twoMer_dict = {**a_dict, **new_mer_dict}
twoMer_dict

{'000': 4,
 '100': 5,
 '110': 6,
 '111': 7,
 '112': 8,
 '113': 9,
 '114': 10,
 '120': 11,
 '121': 12,
 '122': 13,
 '123': 14,
 '124': 15,
 '130': 16,
 '131': 17,
 '132': 18,
 '133': 19,
 '134': 20,
 '140': 21,
 '141': 22,
 '142': 23,
 '143': 24,
 '144': 25,
 '200': 26,
 '210': 27,
 '211': 28,
 '212': 29,
 '213': 30,
 '214': 31,
 '220': 32,
 '221': 33,
 '222': 34,
 '223': 35,
 '224': 36,
 '230': 37,
 '231': 38,
 '232': 39,
 '233': 40,
 '234': 41,
 '240': 42,
 '241': 43,
 '242': 44,
 '243': 45,
 '244': 46,
 '300': 47,
 '310': 48,
 '311': 49,
 '312': 50,
 '313': 51,
 '314': 52,
 '320': 53,
 '321': 54,
 '322': 55,
 '323': 56,
 '324': 57,
 '330': 58,
 '331': 59,
 '332': 60,
 '333': 61,
 '334': 62,
 '340': 63,
 '341': 64,
 '342': 65,
 '343': 66,
 '344': 67,
 '400': 68,
 '410': 69,
 '411': 70,
 '412': 71,
 '413': 72,
 '414': 73,
 '420': 74,
 '421': 75,
 '422': 76,
 '423': 77,
 '424': 78,
 '430': 79,
 '431': 80,
 '432': 81,
 '433': 82,
 '434': 83,
 '440': 84,
 '441': 85,
 '442': 86,
 '443': 87

In [ ]:
class RegularDataset(Dataset):
  def __init__(self, orig_diseases, orig_mirna, orig_labels, hair_x, is_conv1d=False, is_mlp=False):
    super(RegularDataset, self).__init__()
    self.orig_diseases = orig_diseases
    self.orig_mirna = orig_mirna
    self.orig_labels = orig_labels
    self.vocab_size = 5
    self.MAX_LEN = 28
    self.MAX_LEN_D = 6
    self.hair_x = hair_x
    self.is_conv1d = is_conv1d
    self.is_mlp = is_mlp
    self.letter2Number = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "A": 4, "C": 5, "G": 6, "U": 7}

  def convert_letter_to_number(self, lettter):
    letter_to_number = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "A": 4, "C": 5, "G": 6, "U": 7}

    return letter_to_number[lettter]

  def convert_letter_to_number_Conv1D(self, lettter):
    letter_to_number = {'[PAD]': 0, "A": 1, "C": 2, "G": 3, "U": 4}

    return letter_to_number[lettter]

  def make_data_with_unified_length(self, token_list, max_len, covert_dict, is_conv1d=False):
      max_len = max_len + 2  # add [CLS] and [SEP]
      if is_conv1d:
        token_list = token_list
      else:
        token_list = [covert_dict['[CLS]']] + token_list + [covert_dict['[SEP]']]
      
      n_pad = max_len - len(token_list)
      token_list.extend([0] * n_pad)

      return token_list

  def mirna2list(self, data, is_conv1d=False):
    out = []
    if is_conv1d:
      for j in range(len(data)):
        coverted_rna = self.convert_letter_to_number_Conv1D(data[j])
        out.append(coverted_rna)
    else:
      for j in range(len(data)):
        coverted_rna = self.convert_letter_to_number(data[j])
        out.append(coverted_rna)

    data = self.make_data_with_unified_length(out, self.MAX_LEN, self.letter2Number, is_conv1d=is_conv1d)
    
    return np.array(data)

  
  def disease2list(self, disease):
    disease = disease.replace(",", "")
    disease = disease.replace("-", " ")
    disease = disease.replace("[", "")
    disease = disease.replace("]", "")
    disease = disease.replace("Kideny", "Kidney")
    disease = disease.replace("Spondylarthritis", "Spondyloarthritis")
    disease_list = disease.split(" ")
    return disease_list


  def disease2emb(self, disease):

    disease_list = self.disease2list(disease)
    vector = np.zeros(300)
    for word in disease_list:
      value = word2vec.get(word)
      if value is not None:
        vector += word2vec[word]
      else:
        vector = np.zeros(300)
    out = vector/len(disease_list)
    return out

  def disease2token(self, disease):
  
    disease = disease.replace(", ", " ")
    disease_list = disease.split(" ")
    
    # print(disease_list)
    tokenized=[self.disease_token_list[i] for i in disease_list]
    max_len = self.MAX_LEN_D + 2
    tokenized = [self.disease_token_list['[CLS]']] + tokenized + [self.disease_token_list['[SEP]']]
    n_pad = max_len - len(tokenized)
    tokenized.extend([0] * n_pad)
    return np.array(tokenized)

  def __getitem__(self, index):
      orig_disease = self.orig_diseases[index]
      orig_mirna = self.orig_mirna[index]
      label = self.orig_labels[index]+1
      # mirna = self.mirna2list(orig_mirna)
      # mirna = np.array([self.mirna2list(orig_mirna, is_conv1d=self.is_conv1d)])
      if self.is_conv1d:
        mirna = np.array([self.mirna2list(orig_mirna, is_conv1d=self.is_conv1d)])
        mirna = torch.from_numpy(mirna).float()
      else:
        mirna = self.mirna2list(orig_mirna, is_conv1d=self.is_conv1d)
        mirna = np.expand_dims(mirna, axis=0)
      
      if self.is_mlp:
        pre_hair_x = torch.from_numpy(np.array([*self.hair_x[index], *self.mirna2list(orig_mirna)])).float()
      else:
        pre_hair_x = torch.from_numpy(np.array([*self.hair_x[index]])).float()

      disease = self.disease2emb(orig_disease)
      # disease = np.expand_dims(disease, axis=0)
      return disease, mirna, label, pre_hair_x, orig_disease, orig_mirna
      

  def __len__(self):
    return len(self.orig_diseases)


In [ ]:
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
dataloader = DataLoader(dataset, batch_size=2)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(idx)
  print('disease.shape ',disease.shape)
  print('mirna ',mirna)
  print('pre_hair_x.shape ',pre_hair_x.shape)
  print('mirna.shape', mirna.shape)
  print('label.shape ',label.shape)
  print('label ',label)
  print('orig_disease ',orig_disease)
  print('orig_mirna ',orig_mirna)

  break

0
disease.shape  torch.Size([2, 300])
mirna  tensor([[[1, 7, 4, 6, 5, 4, 6, 5, 4, 5, 4, 7, 4, 4, 7, 6, 6, 7, 7, 7, 6, 7, 6,
          2, 0, 0, 0, 0, 0, 0]],

        [[1, 5, 4, 6, 6, 5, 5, 4, 7, 4, 7, 7, 6, 7, 6, 5, 7, 6, 5, 5, 7, 5, 4,
          2, 0, 0, 0, 0, 0, 0]]])
pre_hair_x.shape  torch.Size([2, 41])
mirna.shape torch.Size([2, 1, 30])
label.shape  torch.Size([2])
label  tensor([0.6000, 0.6000], dtype=torch.float64)
orig_disease  ('Leukemia, Lymphocytic, Chronic, B-Cell', 'Leukemia, Lymphocytic, Chronic, B-Cell')
orig_mirna  ('UAGCAGCACAUAAUGGUUUGUG', 'CAGGCCAUAUUGUGCUGCCUCA')


In [ ]:

dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_conv1d=True)
dataloader = DataLoader(dataset, batch_size=2)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(idx)
  print('disease.shape ',disease.shape)
  print('mirna ',mirna)
  print('pre_hair_x.shape ',pre_hair_x.shape)
  print('mirna.shape', mirna.shape)
  print('label.shape ',label.shape)
  print('label ',label)
  print('orig_disease ',orig_disease)
  print('orig_mirna ',orig_mirna)

  break

0
disease.shape  torch.Size([2, 300])
mirna  tensor([[[4., 1., 3., 2., 1., 3., 2., 1., 2., 1., 4., 1., 1., 4., 3., 3., 4.,
          4., 4., 3., 4., 3., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[2., 1., 3., 3., 2., 2., 1., 4., 1., 4., 4., 3., 4., 3., 2., 4., 3.,
          2., 2., 4., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]])
pre_hair_x.shape  torch.Size([2, 41])
mirna.shape torch.Size([2, 1, 30])
label.shape  torch.Size([2])
label  tensor([0.6000, 0.6000], dtype=torch.float64)
orig_disease  ('Leukemia, Lymphocytic, Chronic, B-Cell', 'Leukemia, Lymphocytic, Chronic, B-Cell')
orig_mirna  ('UAGCAGCACAUAAUGGUUUGUG', 'CAGGCCAUAUUGUGCUGCCUCA')


In [ ]:

dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_mlp=True)
dataloader = DataLoader(dataset, batch_size=2)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(idx)
  print('disease.shape ',disease.shape)
  print('mirna ',mirna)
  print('pre_hair_x.shape ',pre_hair_x.shape)
  print('mirna.shape', mirna.shape)
  print('label.shape ',label.shape)
  print('label ',label)
  print('orig_disease ',orig_disease)
  print('orig_mirna ',orig_mirna)

  break

0
disease.shape  torch.Size([2, 300])
mirna  tensor([[[1, 7, 4, 6, 5, 4, 6, 5, 4, 5, 4, 7, 4, 4, 7, 6, 6, 7, 7, 7, 6, 7, 6,
          2, 0, 0, 0, 0, 0, 0]],

        [[1, 5, 4, 6, 6, 5, 5, 4, 7, 4, 7, 7, 6, 7, 6, 5, 7, 6, 5, 5, 7, 5, 4,
          2, 0, 0, 0, 0, 0, 0]]])
pre_hair_x.shape  torch.Size([2, 71])
mirna.shape torch.Size([2, 1, 30])
label.shape  torch.Size([2])
label  tensor([0.6000, 0.6000], dtype=torch.float64)
orig_disease  ('Leukemia, Lymphocytic, Chronic, B-Cell', 'Leukemia, Lymphocytic, Chronic, B-Cell')
orig_mirna  ('UAGCAGCACAUAAUGGUUUGUG', 'CAGGCCAUAUUGUGCUGCCUCA')


# Model

## LSTM

In [ ]:
class LSTM_module(nn.Module):
  def __init__(self):
    super(LSTM_module, self).__init__()
    self.hidden_size = 64
    self.embedding_dim = 30
    self.num_layer = 2
    self.bidirectional = True
    self.bi_num = 2 if self.bidirectional else 1
    self.dropout = 0.5
    # input(seq_len, batch, input_size)
    # h0(num_layers * num_directions, batch, hidden_size)
    # c0(num_layers * num_directions, batch, hidden_size)
    self.lstm1 = nn.LSTM(self.embedding_dim, self.hidden_size, self.num_layer, bidirectional=True, dropout=self.dropout)
    # self.lstm2 = nn.LSTM(128, 64, self.num_layer, bidirectional=True, dropout=self.dropout)
    self.FC = nn.Linear(128, 64)

  def forward(self, x):
    h_0, c_0 = self.init_hidden_state(x.size(0))
    # x = np.transpose(x, (1,0,2))
    x = torch.swapaxes(x, 0, 1)
    x = x.to(torch.float32)
    # print('h_0', h_0.dtype)
    # print('c_0', c_0.dtype)
    # print('x', x.dtype)
    x, (h_n, c_n) = self.lstm1(x, (h_0, c_0))
    # print('x', x.shape)
    # x, (h_n, c_n) = self.lstm2(x, (h_n, c_n))
    # print('output', output)
    # print('h_n', h_n.shape)
    # print('c_n', c_n.shape)
    out = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=-1)
    # print(out)
    out = self.FC(out)
    return out
  
  def init_hidden_state(self, batch_size):
    h_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    c_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    return h_0, c_0

In [ ]:
class Classificaion_module(nn.Module):
    def __init__(self, tanhshrink_ratio=3):
      super(Classificaion_module, self).__init__()
      self.tanhshrink_ratio = tanhshrink_ratio
      self.lstm_module = LSTM_module()
      self.mlp_sec = nn.Sequential(
          nn.Linear(41, 64),
          nn.BatchNorm1d(64),
          nn.ReLU()
      )
      self.mlp = nn.Sequential(
          nn.Linear(428, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          nn.Dropout(0.4),
          # nn.Linear(128, 64),
          # nn.BatchNorm1d(64),
          # nn.ReLU(),
          nn.Linear(128, 32),
          nn.Dropout(0.2),
          nn.BatchNorm1d(32),
          nn.ReLU(),
          nn.Linear(32, 1)
      )
      

    def forward(self, x_d, x_mr, x_sec):
      d_mr = self.lstm_module(x_mr)
      d_sec = self.mlp_sec(x_sec)
      # print('x_d.shape',x_d.shape)
      # print('x_mr.shape',x_mr.shape)
      # print('d_d.shape',d_d.shape)
      # print('d_mr.shape',d_mr.shape)
      x = torch.cat((d_mr, d_sec, x_d), dim=1)
      # print(x.shape)
      x = self.mlp(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-960, 960])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=960)
val_dataloader = DataLoader(val_dataset, batch_size=500)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(mirna.shape)
  mirna = mirna.cuda()
  disease = disease.to(torch.float32).cuda()
  secondary_feature = pre_hair_x.cuda()
  print(disease.shape)
  print(secondary_feature.shape)
  output = model_class(disease, mirna, secondary_feature)
  print('output.shape', output.shape)

  break

torch.Size([32, 1, 30])
torch.Size([32, 300])
torch.Size([32, 41])
output.shape torch.Size([32, 1])


## GRU

In [ ]:
class GRU_module(nn.Module):
  def __init__(self):
    super(GRU_module, self).__init__()
    self.hidden_size = 64
    self.embedding_dim = 30
    self.num_layer = 2
    self.bidirectional = True
    self.bi_num = 2 if self.bidirectional else 1
    self.dropout = 0.5
    # input(seq_len, batch, input_size)
    # h0(num_layers * num_directions, batch, hidden_size)
    # c0(num_layers * num_directions, batch, hidden_size)
    self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.num_layer, bidirectional=True, dropout=self.dropout)
    # self.lstm2 = nn.LSTM(128, 64, self.num_layer, bidirectional=True, dropout=self.dropout)
    self.FC = nn.Linear(128, 64)

  def forward(self, x):
    h_0, c_0 = self.init_hidden_state(x.size(0))
    # x = np.transpose(x, (1,0,2))
    x = torch.swapaxes(x, 0, 1)
    x = x.to(torch.float32)
    # print('h_0', h_0.dtype)
    # print('c_0', c_0.dtype)
    # print('x', x.dtype)
    x, h_n = self.gru(x, h_0)
    # print('x', x.shape)
    # x, (h_n, c_n) = self.lstm2(x, (h_n, c_n))
    # print('output', output)
    # print('h_n', h_n.shape)
    # print('c_n', c_n.shape)
    out = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=-1)
    # print(out)
    out = self.FC(out)
    return out
  
  def init_hidden_state(self, batch_size):
    h_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    c_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    return h_0, c_0

In [ ]:
class Classificaion_module(nn.Module):
    def __init__(self, tanhshrink_ratio=3):
      super(Classificaion_module, self).__init__()
      self.tanhshrink_ratio = tanhshrink_ratio
      self.gru_module = GRU_module()
      self.mlp_sec = nn.Sequential(
          nn.Linear(41, 64),
          nn.BatchNorm1d(64),
          nn.ReLU()
      )
      self.mlp = nn.Sequential(
          nn.Linear(428, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          # nn.Linear(128, 64),
          # nn.BatchNorm1d(64),
          # nn.ReLU(),
          nn.Linear(128, 32),
          nn.BatchNorm1d(32),
          nn.ReLU(),
          nn.Linear(32, 1)
      )
      

    def forward(self, x_d, x_mr, x_sec):
      d_mr = self.gru_module(x_mr)
      d_sec = self.mlp_sec(x_sec)
      # print('x_d.shape',x_d.shape)
      # print('x_mr.shape',x_mr.shape)
      # print('d_d.shape',d_d.shape)
      # print('d_mr.shape',d_mr.shape)
      x = torch.cat((d_mr, d_sec, x_d), dim=1)
      # print(x.shape)
      x = self.mlp(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-960, 960])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=960)
val_dataloader = DataLoader(val_dataset, batch_size=500)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(mirna.shape)
  mirna = mirna.cuda()
  disease = disease.to(torch.float32).cuda()
  secondary_feature = pre_hair_x.cuda()
  print(disease.shape)
  print(secondary_feature.shape)
  output = model_class(disease, mirna, secondary_feature)
  print('output.shape', output.shape)

  break

torch.Size([32, 1, 30])
torch.Size([32, 300])
torch.Size([32, 41])
output.shape torch.Size([32, 1])


## BERT

In [ ]:
def get_train_config():
    parse = argparse.ArgumentParser(description='iDNA_ABT train model')

    # preoject setting
    parse.add_argument('-learn-name', type=str, default='iDNA_ABT_train', help='learn name')
    parse.add_argument('-save-best', type=bool, default=False, help='if save parameters of the current best model ')
    parse.add_argument('-threshold', type=float, default=0.80, help='save threshold')
    
    # model parameters
    parse.add_argument('-vocab_size', type=int, default=8, help='dictionary size of letter to token')
    parse.add_argument('-max-len', type=int, default=30, help='max length of input sequences')
    parse.add_argument('-num-layer', type=int, default=3, help='number of encoder blocks')  # 3
    parse.add_argument('-num-head', type=int, default=8, help='number of head in multi-head attention')  # 8
    parse.add_argument('-dim-embedding', type=int, default=64, help='residue embedding dimension')  # 64
    parse.add_argument('-dim-feedforward', type=int, default=64, help='hidden layer dimension in feedforward layer')
    parse.add_argument('-dim-k', type=int, default=32, help='embedding dimension of vector k or q')
    parse.add_argument('-dim-v', type=int, default=32, help='embedding dimension of vector v')
    parse.add_argument('-num-embedding', type=int, default=2, help='number of sense in multi-sense')
    parse.add_argument('-k-mer', type=int, default=3, help='number of k(-mer) in multi-sccaled')
    parse.add_argument('-embed-atten-size', type=int, default=8, help='size of soft attention')

    # training parameters
    parse.add_argument('-lr', type=float, default=0.0005, help='learning rate')
    parse.add_argument('-reg', type=float, default=0.0025, help='weight lambda of regularization')
    parse.add_argument('-batch-size', type=int, default=64, help='number of samples in a batch')
    parse.add_argument('-epoch', type=int, default=100, help='number of iteration')  # 30
    parse.add_argument('-k-fold', type=int, default=-1, help='k in cross validation,-1 represents train-test approach')
    parse.add_argument('-num-class', type=int, default=3, help='number of classes')
    parse.add_argument('-b', type=int, default=0.06, help='b')
    # parse.add_argument('-cuda', type=bool, default=True, help='if use cuda')
    parse.add_argument('-cuda', type=bool, default=False, help='if not use cuda')
    parse.add_argument('-device', type=int, default=0, help='device id')
    parse.add_argument('-interval-log', type=int, default=10,
                       help='how many batches have gone through to record the training performance')
    parse.add_argument('-interval-valid', type=int, default=5,
                       help='how many epoches have gone through to record the validation performance')  # 20
    parse.add_argument('-interval-test', type=int, default=1,
                       help='how many epoches have gone through to record the test performance')
    parse.add_argument('-alpha', type=float, default=0.1, help='information entropy')

    config = parse.parse_args(args=[])

    return config

In [ ]:
def set_freeze_by_names(model, layer_names, freeze=True):
    if not isinstance(layer_names, Iterable):
        layer_names = [layer_names]
    for name, child in model.named_children():
        if name not in layer_names:
            continue
        for param in child.parameters():
            param.requires_grad = not freeze


def freeze_by_names(model, layer_names):
    set_freeze_by_names(model, layer_names, True)


def unfreeze_by_names(model, layer_names):
    set_freeze_by_names(model, layer_names, False)


def set_freeze_by_idxs(model, idxs, freeze=True):
    if not isinstance(idxs, Iterable):
        idxs = [idxs]
    num_child = len(list(model.children()))
    idxs = tuple(map(lambda idx: num_child + idx if idx < 0 else idx, idxs))
    for idx, child in enumerate(model.children()):
        if idx not in idxs:
            continue
        for param in child.parameters():
            param.requires_grad = not freeze


def freeze_by_idxs(model, idxs):
    set_freeze_by_idxs(model, idxs, True)


def unfreeze_by_idxs(model, idxs):
    set_freeze_by_idxs(model, idxs, False)

In [ ]:
def get_attn_pad_mask(seq):
    batch_size, seq_len = seq.size()
    pad_attn_mask = seq.data.eq(0).unsqueeze(1)  # [batch_size, 1, seq_len]
    pad_attn_mask_expand = pad_attn_mask.expand(batch_size, seq_len, seq_len)  # [batch_size, seq_len, seq_len]
    return pad_attn_mask_expand


class Embedding(nn.Module):
    def __init__(self, config):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding (look-up table) [64, 43, 64]
        self.pos_embed = nn.Embedding(max_len, d_model)  # position embedding [64, 43, 64]
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        seq_len = x.size(1)  # x: [batch_size, seq_len]
        
        pos = torch.arange(seq_len, device=device, dtype=torch.long)  # [seq_len]

        pos = pos.unsqueeze(0).expand_as(x)  # [seq_len] -> [batch_size, seq_len]
        embedding = self.pos_embed(pos)
        embedding = embedding + self.tok_embed(x)
        embedding = self.norm(embedding)
        return embedding


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)  # scores : [batch_size, n_head, seq_len, seq_len]
        scores.masked_fill_(attn_mask, -1e9)  # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)  # [batch_size, n_head, seq_len, seq_len]
        context = torch.matmul(attn, V)  # [batch_size, n_head, seq_len, d_v]
        return context, attn


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_head)
        self.W_K = nn.Linear(d_model, d_k * n_head)
        self.W_V = nn.Linear(d_model, d_v * n_head)

        self.linear = nn.Linear(n_head * d_v, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_head, d_k).transpose(1, 2)  # q_s: [batch_size, n_head, seq_len, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_head, d_k).transpose(1, 2)  # k_s: [batch_size, n_head, seq_len, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_head, d_v).transpose(1, 2)  # v_s: [batch_size, n_head, seq_len, d_v]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_head, 1, 1)
        context, attention_map = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1,
                                                            n_head * d_v)  # context: [batch_size, seq_len, n_head * d_v]
        output = self.linear(context)
        output = self.norm(output + residual)
        return output, attention_map


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        return self.fc2(self.relu(self.fc1(x)))


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
        self.attention_map = None

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attention_map = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs,
                                                        enc_self_attn_mask)  # enc_inputs to same Q,K,V
        self.attention_map = attention_map
        enc_outputs = self.pos_ffn(enc_outputs)  # enc_outputs: [batch_size, seq_len, d_model]
        return enc_outputs


class BERT(nn.Module):
    def __init__(self, config):
        super(BERT, self).__init__()

        global max_len, n_layers, n_head, d_model, d_ff, d_k, d_v, vocab_size, device
        max_len = config.max_len
        n_layers = config.num_layer
        n_head = config.num_head
        d_model = config.dim_embedding
        d_ff = config.dim_feedforward
        d_k = config.dim_k
        d_v = config.dim_v
        vocab_size = config.vocab_size
        device = torch.device("cuda")

        self.embedding = Embedding(config)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc_task = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(d_model // 2, 2),
        )
        self.classifier = nn.Linear(2, 2)

    def forward(self, input_ids):
        output = self.embedding(input_ids)  # [bach_size, seq_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(input_ids)  # [batch_size, maxlen, maxlen]
        for layer in self.layers:
            output = layer(output, enc_self_attn_mask)
            # output: [batch_size, max_len, d_model]

        # classification
        # only use [CLS]
        representation = output[:, 0, :]
        # print(representation.shape)
        reduction_feature = self.fc_task(representation)
        reduction_feature = reduction_feature.view(reduction_feature.size(0), -1)
        logits_clsf = self.classifier(reduction_feature)
        # representation = reduction_feature
        return logits_clsf, representation

In [ ]:
class Classificaion_module(nn.Module):
    def __init__(self, config_mrna, tanhshrink_ratio=3):
      super(Classificaion_module, self).__init__()
      self.tanhshrink_ratio = tanhshrink_ratio
      self.bert_module = BERT(config_mrna)
      self.mlp_sec = nn.Sequential(
          nn.Linear(41, 64),
          nn.BatchNorm1d(64),
          nn.ReLU()
      )
      self.mlp = nn.Sequential(
          nn.Linear(428, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          # nn.Linear(128, 64),
          # nn.BatchNorm1d(64),
          # nn.ReLU(),
          nn.Linear(128, 32),
          nn.BatchNorm1d(32),
          nn.ReLU(),
          nn.Linear(32, 1)
      )
      

    def forward(self, x_d, x_mr, x_sec):
      x_mr = x_mr.squeeze()
      _, d_mr = self.bert_module(x_mr)
      d_sec = self.mlp_sec(x_sec)
      # print('x_d.shape',x_d.shape)
      # print('x_mr.shape',x_mr.shape)
      # print('d_d.shape',d_d.shape)
      # print('d_mr.shape',d_mr.shape)
      x = torch.cat((d_mr, d_sec, x_d), dim=1)
      # print(x.shape)
      x = self.mlp(x)

      return x



In [ ]:
config_mrna = get_train_config()
model_class = Classificaion_module(config_mrna).cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-960, 960])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=960)
val_dataloader = DataLoader(val_dataset, batch_size=500)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(mirna.shape)
  mirna = mirna.cuda()
  disease = disease.to(torch.float32).cuda()
  secondary_feature = pre_hair_x.cuda()
  print(disease.shape)
  print(secondary_feature.shape)
  output = model_class(disease, mirna, secondary_feature)
  print('output.shape', output.shape)

  break

torch.Size([32, 1, 30])
torch.Size([32, 300])
torch.Size([32, 41])
output.shape torch.Size([32, 1])


## Conv1d

In [ ]:
class Conv1d_module(torch.nn.Module):
    def __init__(self):
        super(Conv1d_module, self).__init__()
        self.conv1d_1 = nn.Conv1d(1, 1, 3, stride=1, padding=1)
        self.batchnorm_1 = nn.BatchNorm1d(1)
        self.conv1d_2 = nn.Conv1d(1, 2, 3, stride=1, padding=1)
        self.batchnorm_2 = nn.BatchNorm1d(2)
        self.conv1d_3 = nn.Conv1d(2, 4, 3, stride=1, padding=1)
        self.batchnorm_3 = nn.BatchNorm1d(4)
        self.linear_1 = nn.Linear(120, 64)
        self.relu = nn.ReLU()
        
        # self.linear_2 = nn.Linear(320, 64)

    def forward(self, x):
        x = self.conv1d_1(x)
        x = self.batchnorm_1(x)
        x = self.relu(x)
        x = self.conv1d_2(x)
        x = self.batchnorm_2(x)
        x = self.relu(x)
        x = self.conv1d_3(x)
        x = self.batchnorm_3(x)
        x = self.relu(x)
        x = torch.flatten(x,start_dim=1)
        # print(x.shape)
        out = self.linear_1(x)
        # out = self.linear_2(x)

        

        return out



In [ ]:
class Classificaion_module(nn.Module):
    def __init__(self, tanhshrink_ratio=3):
      super(Classificaion_module, self).__init__()
      self.tanhshrink_ratio = tanhshrink_ratio
      self.conv1d_module = Conv1d_module()
      self.mlp_sec = nn.Sequential(
          nn.Linear(41, 64),
          nn.BatchNorm1d(64),
          nn.ReLU()
      )
      self.mlp = nn.Sequential(
          nn.Linear(428, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          # nn.Linear(128, 64),
          # nn.BatchNorm1d(64),
          # nn.ReLU(),
          nn.Linear(128, 32),
          nn.BatchNorm1d(32),
          nn.ReLU(),
          nn.Linear(32, 1)
      )
      

    def forward(self, x_d, x_mr, x_sec):

      d_mr = self.conv1d_module(x_mr)
      d_sec = self.mlp_sec(x_sec)
      # print('x_d.shape',x_d.shape)
      # print('x_mr.shape',x_mr.shape)
      # print('d_d.shape',d_d.shape)
      # print('d_mr.shape',d_mr.shape)
      x = torch.cat((d_mr, d_sec, x_d), dim=1)
      # print(x.shape)
      x = self.mlp(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_conv1d=True)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-960, 960])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=960)
val_dataloader = DataLoader(val_dataset, batch_size=500)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(mirna.shape)
  
  mirna = mirna.cuda()

  disease = disease.to(torch.float32).cuda()
  secondary_feature = pre_hair_x.cuda()
  print(disease.shape)
  print(secondary_feature.shape)
  output = model_class(disease, mirna, secondary_feature)
  print('output.shape', output.shape)

  break

torch.Size([32, 1, 30])
torch.Size([32, 300])
torch.Size([32, 41])
output.shape torch.Size([32, 1])


## MLP

In [ ]:
# from torch.nn.modules.batchnorm import BatchNorm1d
class Classificaion_module(nn.Module):
    def __init__(self, tanhshrink_ratio=3):
      super(Classificaion_module, self).__init__()
      self.tanhshrink_ratio = tanhshrink_ratio

      # self.conv1d_module = Conv1d_module()
      self.linear_1 = nn.Linear(71, 128)
      self.bn_1 = nn.BatchNorm1d(128)
      self.linear_2 = nn.Linear(128, 256)
      self.bn_2 = nn.BatchNorm1d(256)
      self.linear_3 = nn.Linear(256, 128)
      self.bn_3 = nn.BatchNorm1d(128)
      self.linear_4 = nn.Linear(128, 64)
      self.bn_4 = nn.BatchNorm1d(64)
      self.dropout = nn.Dropout(0.2)
      self.RELU = nn.ReLU()
      self.FC = nn.Sequential(
          nn.Linear(364, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.BatchNorm1d(64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.BatchNorm1d(32),
          nn.ReLU(),
          nn.Linear(32, 1),

      )

      


    def forward(self, x_d, pre_hair_x):
      # print('x_d type',type(x_d))
      # print('x_mr type',type(x_mr))
      # d_mr = self.conv1d_module(data.mirna)
      # print('data.pre_hair_x shape ', data.pre_hair_x.shape)
      d_mr = self.linear_1(pre_hair_x)
      d_mr = self.bn_1(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_2(d_mr)
      d_mr = self.bn_2(d_mr)
      d_mr = self.RELU(d_mr)
      d_mr = self.dropout(d_mr)
      d_mr = self.linear_3(d_mr)
      d_mr = self.bn_3(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_4(d_mr)
      d_mr = self.bn_4(d_mr)
      d_mr = self.RELU(d_mr)

      x = torch.cat((x_d, d_mr), dim=1)
      # print(x.shape)
      x = self.FC(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_mlp=True)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-960, 960])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=960)
val_dataloader = DataLoader(val_dataset, batch_size=500)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(mirna.shape)
  
  mirna = mirna.cuda()

  disease = disease.to(torch.float32).cuda()
  secondary_feature = pre_hair_x.cuda()
  print(disease.shape)
  print(secondary_feature.shape)
  output = model_class(disease, secondary_feature)
  print('output.shape', output.shape)

  break

torch.Size([32, 1, 30])
torch.Size([32, 300])
torch.Size([32, 71])
output.shape torch.Size([32, 1])


# Train

## LSTM

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (lstm_module): LSTM_module(
    (lstm1): LSTM(30, 64, num_layers=2, dropout=0.5, bidirectional=True)
    (FC): Linear(in_features=128, out_features=64, bias=True)
  )
  (mlp_sec): Sequential(
    (0): Linear(in_features=41, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (mlp): Sequential(
    (0): Linear(in_features=428, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=32, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    digits = model_class(disease, mirna, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    mirna = mirna.cuda()
    digits = model_class(disease, mirna, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTraining Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(300)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Training Epoch[0] Loss:0.4059975612908602 | L1 Loss:0.4932502768933773 | R2:-0.14056239216215197 | ACC: 58.8000%(294/500)
Testing Epoch[0] Loss:0.4300575256347656 | L1 Loss:0.5099345415830612 | R2:-0.1351200990163987 | ACC: 60.0000%(180/300)
Training Epoch[1] Loss:0.38072561752051115 | L1 Loss:0.47631519474089146 | R2:-0.05111064050740481 | ACC: 61.6000%(308/500)
Testing Epoch[1] Loss:0.4047554641962051 | L1 Loss:0.4902379661798477 | R2:-0.07592985481530609 | ACC: 61.0000%(183/300)
Training Epoch[2] Loss:0.3634476652368903 | L1 Loss:0.4574627336114645 | R2:-0.009777736680132106 | ACC: 62.4000%(312/500)
Testing Epoch[2] Loss:0.37098280489444735 | L1 Loss:0.4677279472351074 | R2:0.01763742902653128 | ACC: 63.3333%(190/300)
Training Epoch[3] Loss:0.3339084777981043 | L1 Loss:0.43487480469048023 | R2:0.07926221890893953 | ACC: 63.4000%(317/500)
Testing Epoch[3] Loss:0.3681647926568985 | L1 Loss:0.45198107063770293 | R2:0.023688621919907716 | ACC: 64.3333%(193/300)
Training Epoch[4] Loss:0.

In [ ]:
print('-----LSTM+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))

-----LSTM+ MLP 300 epochs-----
max test_r2_record  0.36779682975069206
min test_loss_l1_record  0.36098951995372774
min test_loss_record  0.23597124963998795


In [ ]:
print('-----LSTM+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----LSTM+ MLP 300 epochs-----
max test_r2_record  0.4539764606567404
min test_loss_l1_record  0.34505664110183715
min test_loss_record  0.19740368351340293


In [ ]:
print('-----LSTM+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----LSTM+ MLP 100 epochs-----
max test_r2_record  0.40889370284073223
min test_loss_l1_record  0.34584062695503237
min test_loss_record  0.21329353600740433


In [ ]:
print('-----LSTM+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----LSTM+ MLP 50 epochs-----
max test_r2_record  0.33985270740833773
min test_loss_l1_record  0.37808668315410615
min test_loss_record  0.23791675567626952


In [ ]:
print('-----LSTM+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----LSTM+ MLP 300 epochs-----
max test_r2_record  0.55865865548055
min test_loss_l1_record  0.298935204744339
min test_loss_record  0.18107207119464874


In [ ]:
print('-----LSTM+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----LSTM+ MLP 100 epochs-----
max test_r2_record  0.5205038869404067
min test_loss_l1_record  0.31958115100860596
min test_loss_record  0.19672611355781555


In [ ]:
print('-----LSTM+ MLP 200 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:200]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:200]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:200]))

-----LSTM+ MLP 200 epochs-----
max test_r2_record  0.552302451566165
min test_loss_l1_record  0.3034108579158783
min test_loss_record  0.18367987871170044


In [ ]:
print('-----LSTM+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----LSTM+ MLP 50 epochs-----
max test_r2_record  0.45373308581377814
min test_loss_l1_record  0.3382137417793274
min test_loss_record  0.2241206020116806


## GRU

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (gru_module): GRU_module(
    (gru): GRU(30, 64, num_layers=2, dropout=0.5, bidirectional=True)
    (FC): Linear(in_features=128, out_features=64, bias=True)
  )
  (mlp_sec): Sequential(
    (0): Linear(in_features=41, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (mlp): Sequential(
    (0): Linear(in_features=428, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=32, bias=True)
    (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    digits = model_class(disease, mirna, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    mirna = mirna.cuda()
    digits = model_class(disease, mirna, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(200)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.3957033306360245 | L1 Loss:0.47571220993995667 | R2:-0.04122904508350231 | ACC: 59.0000%(295/500)
Testing Epoch[0] Loss:0.3858307167887688 | L1 Loss:0.46802532970905303 | R2:-0.012582346996903781 | ACC: 63.0000%(189/300)
Testing Epoch[1] Loss:0.35906716994941235 | L1 Loss:0.45285539887845516 | R2:0.060582040947702157 | ACC: 61.8000%(309/500)
Testing Epoch[1] Loss:0.35153576731681824 | L1 Loss:0.44123417139053345 | R2:0.0765907289610338 | ACC: 63.6667%(191/300)
Testing Epoch[2] Loss:0.3298210445791483 | L1 Loss:0.436803026124835 | R2:0.14093842215103608 | ACC: 62.0000%(310/500)
Testing Epoch[2] Loss:0.34147956073284147 | L1 Loss:0.4347683846950531 | R2:0.09923356579213696 | ACC: 67.3333%(202/300)
Testing Epoch[3] Loss:0.31075404305011034 | L1 Loss:0.42227941006422043 | R2:0.1893496411900744 | ACC: 63.6000%(318/500)
Testing Epoch[3] Loss:0.335654978454113 | L1 Loss:0.4304762721061707 | R2:0.11969857233735767 | ACC: 69.0000%(207/300)
Testing Epoch[4] Loss:0.2908077

In [ ]:
print('-----GRU+ MLP 200 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----GRU+ MLP 200 epochs-----
max test_r2_record  0.37302114988344404
min test_loss_l1_record  0.3450680568814278
min test_loss_record  0.23189863041043282


In [ ]:
print('-----GRU+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----GRU+ MLP 100 epochs-----
max test_r2_record  0.3202273865438901
min test_loss_l1_record  0.36992483735084536
min test_loss_record  0.2564698293805122


In [ ]:
print('-----GRU+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----GRU+ MLP 50 epochs-----
max test_r2_record  0.28181404451967007
min test_loss_l1_record  0.3914389729499817
min test_loss_record  0.2684503376483917


## BERT

In [ ]:
config_mrna = get_train_config()
model_class = Classificaion_module(config_mrna).cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (bert_module): BERT(
    (embedding): Embedding(
      (tok_embed): Embedding(8, 64)
      (pos_embed): Embedding(30, 64)
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(in_features=64, out_features=256, bias=True)
          (W_K): Linear(in_features=64, out_features=256, bias=True)
          (W_V): Linear(in_features=64, out_features=256, bias=True)
          (linear): Linear(in_features=256, out_features=64, bias=True)
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ffn): PoswiseFeedForwardNet(
          (fc1): Linear(in_features=64, out_features=64, bias=True)
          (fc2): Linear(in_features=64, out_features=64, bias=True)
          (relu): ReLU()
        )
      )
      (1): EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(in_feature

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    digits = model_class(disease, mirna, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    mirna = mirna.cuda()
    digits = model_class(disease, mirna, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(200)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.3247112249955535 | L1 Loss:0.42460986971855164 | R2:0.1437690896913646 | ACC: 62.6000%(313/500)
Testing Epoch[0] Loss:0.4063527524471283 | L1 Loss:0.48019022941589357 | R2:0.011191200061615825 | ACC: 58.3333%(175/300)
Testing Epoch[1] Loss:0.2979937558993697 | L1 Loss:0.408096881583333 | R2:0.21273461537958555 | ACC: 65.8000%(329/500)
Testing Epoch[1] Loss:0.3881159335374832 | L1 Loss:0.4692078113555908 | R2:0.05238336934526059 | ACC: 59.6667%(179/300)
Testing Epoch[2] Loss:0.27653005812317133 | L1 Loss:0.3954276368021965 | R2:0.27357330531968815 | ACC: 67.6000%(338/500)
Testing Epoch[2] Loss:0.3704243928194046 | L1 Loss:0.46286865174770353 | R2:0.09767300242000274 | ACC: 61.3333%(184/300)
Testing Epoch[3] Loss:0.2585213938727975 | L1 Loss:0.38693030923604965 | R2:0.32119173199035594 | ACC: 69.4000%(347/500)
Testing Epoch[3] Loss:0.3516015112400055 | L1 Loss:0.450244602560997 | R2:0.1434514523989164 | ACC: 63.3333%(190/300)
Testing Epoch[4] Loss:0.2423085719347 

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
# train_loss_record = []
# train_loss_l1_record = []

# val_acc_record = []
# val_loss_record = []
# val_r2_record = []
# val_loss_l1_record = []

# test_acc_record = []
# test_loss_record = []
# test_r2_record = []
# test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(100)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.13950825296342373 | L1 Loss:0.2816904103383422 | R2:0.6210817801373295 | ACC: 84.6000%(423/500)
Testing Epoch[0] Loss:0.24421013444662093 | L1 Loss:0.37415973246097567 | R2:0.4066150293503135 | ACC: 72.6667%(218/300)
Testing Epoch[1] Loss:0.14090487640351057 | L1 Loss:0.28125882986932993 | R2:0.6195254275983637 | ACC: 83.6000%(418/500)
Testing Epoch[1] Loss:0.2406184509396553 | L1 Loss:0.3744694471359253 | R2:0.4134224711135509 | ACC: 72.0000%(216/300)
Testing Epoch[2] Loss:0.14462526794523 | L1 Loss:0.2903771474957466 | R2:0.6113558475648112 | ACC: 83.0000%(415/500)
Testing Epoch[2] Loss:0.23318011164665223 | L1 Loss:0.36509264409542086 | R2:0.4322389624642134 | ACC: 71.6667%(215/300)
Testing Epoch[3] Loss:0.14924994204193354 | L1 Loss:0.29535956867039204 | R2:0.5981830381550708 | ACC: 81.8000%(409/500)
Testing Epoch[3] Loss:0.2340471178293228 | L1 Loss:0.36813656985759735 | R2:0.43059603379584377 | ACC: 71.0000%(213/300)
Testing Epoch[4] Loss:0.148725641425699

In [ ]:
print('-----BERT+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----BERT+ MLP 300 epochs-----
max test_r2_record  0.46265232316500005
min test_loss_l1_record  0.35957983136177063
min test_loss_record  0.22330205738544465


In [ ]:
print('-----BERT+ MLP 200 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----BERT+ MLP 200 epochs-----
max test_r2_record  0.4462282091191341
min test_loss_l1_record  0.364898481965065
min test_loss_record  0.23000905215740203


In [ ]:
print('-----BERT+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----BERT+ MLP 100 epochs-----
max test_r2_record  0.4034281074029506
min test_loss_l1_record  0.3837520211935043
min test_loss_record  0.24860855489969252


In [ ]:
print('-----BERT+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----BERT+ MLP 50 epochs-----
max test_r2_record  0.36623230131394
min test_loss_l1_record  0.3920943856239319
min test_loss_record  0.26455709636211394


## Conv1d

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_conv1d=True)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (conv1d_module): Conv1d_module(
    (conv1d_1): Conv1d(1, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (batchnorm_1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1d_2): Conv1d(1, 2, kernel_size=(3,), stride=(1,), padding=(1,))
    (batchnorm_2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1d_3): Conv1d(2, 4, kernel_size=(3,), stride=(1,), padding=(1,))
    (batchnorm_3): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear_1): Linear(in_features=120, out_features=64, bias=True)
    (relu): ReLU()
  )
  (mlp_sec): Sequential(
    (0): Linear(in_features=41, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (mlp): Sequential(
    (0): Linear(in_features=428, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    digits = model_class(disease, mirna, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    mirna = mirna.cuda()
    digits = model_class(disease, mirna, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTraining Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(200)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.36702448781579733 | L1 Loss:0.45939972065389156 | R2:0.08924380006036996 | ACC: 59.4000%(297/500)
Testing Epoch[0] Loss:0.3764041155576706 | L1 Loss:0.45881304144859314 | R2:0.038080211995390564 | ACC: 60.0000%(180/300)
Testing Epoch[1] Loss:0.32636731397360563 | L1 Loss:0.4391866121441126 | R2:0.18327849323163625 | ACC: 63.0000%(315/500)
Testing Epoch[1] Loss:0.3621054351329803 | L1 Loss:0.459523594379425 | R2:0.07369013037055577 | ACC: 60.0000%(180/300)
Testing Epoch[2] Loss:0.2943747378885746 | L1 Loss:0.41690921783447266 | R2:0.26456024718187343 | ACC: 65.2000%(326/500)
Testing Epoch[2] Loss:0.3572265043854713 | L1 Loss:0.4603163182735443 | R2:0.08615605220768437 | ACC: 60.6667%(182/300)
Testing Epoch[3] Loss:0.27971675898879766 | L1 Loss:0.4071177467703819 | R2:0.30047380383970085 | ACC: 68.6000%(343/500)
Testing Epoch[3] Loss:0.3610179856419563 | L1 Loss:0.4621302425861359 | R2:0.07143303122870255 | ACC: 61.3333%(184/300)
Testing Epoch[4] Loss:0.2587257586

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
# train_loss_record = []
# train_loss_l1_record = []

# val_acc_record = []
# val_loss_record = []
# val_r2_record = []
# val_loss_l1_record = []

# test_acc_record = []
# test_loss_record = []
# test_r2_record = []
# test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(100)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, mirna, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.14997074706479907 | L1 Loss:0.3028674805536866 | R2:0.6145739861934961 | ACC: 80.8000%(404/500)
Testing Epoch[0] Loss:0.3130040973424911 | L1 Loss:0.4351896971464157 | R2:0.21279177765598845 | ACC: 67.0000%(201/300)
Testing Epoch[1] Loss:0.147736472543329 | L1 Loss:0.2990090372040868 | R2:0.6216583333089616 | ACC: 80.4000%(402/500)
Testing Epoch[1] Loss:0.3129752531647682 | L1 Loss:0.4345860332250595 | R2:0.2088538351472132 | ACC: 66.3333%(199/300)
Testing Epoch[2] Loss:0.14597220160067081 | L1 Loss:0.3000742308795452 | R2:0.6265223390822223 | ACC: 81.6000%(408/500)
Testing Epoch[2] Loss:0.31129287332296374 | L1 Loss:0.4314689815044403 | R2:0.2127506768991334 | ACC: 67.0000%(201/300)
Testing Epoch[3] Loss:0.14482517819851637 | L1 Loss:0.2976503847166896 | R2:0.6329657753538975 | ACC: 82.2000%(411/500)
Testing Epoch[3] Loss:0.3162830501794815 | L1 Loss:0.43332878351211546 | R2:0.19794078188592273 | ACC: 65.6667%(197/300)
Testing Epoch[4] Loss:0.14123255759477615 

In [ ]:
print('-----Conv1d+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----Conv1d+ MLP 300 epochs-----
max test_r2_record  0.2696013764214906
min test_loss_l1_record  0.4075911283493042
min test_loss_record  0.2903534650802612


In [ ]:
print('-----Conv1d+ MLP 200 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----Conv1d+ MLP 200 epochs-----
max test_r2_record  0.2268296748256234
min test_loss_l1_record  0.42276514172554014
min test_loss_record  0.3034987524151802


In [ ]:
print('-----Conv1d+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----Conv1d+ MLP 100 epochs-----
max test_r2_record  0.22000390897115057
min test_loss_l1_record  0.42587103247642516
min test_loss_record  0.3102374985814095


In [ ]:
print('-----Conv1d+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----Conv1d+ MLP 50 epochs-----
max test_r2_record  0.14085059173894407
min test_loss_l1_record  0.4375958025455475
min test_loss_record  0.33504715859889983


## MLP

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, is_mlp=True)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (linear_1): Linear(in_features=71, out_features=128, bias=True)
  (bn_1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_2): Linear(in_features=128, out_features=256, bias=True)
  (bn_2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_3): Linear(in_features=256, out_features=128, bias=True)
  (bn_3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_4): Linear(in_features=128, out_features=64, bias=True)
  (bn_4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (RELU): ReLU()
  (FC): Sequential(
    (0): Linear(in_features=364, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): BatchNorm1d(64, eps=1e-05, mome

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    # mirna = mirna.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    # mirna = mirna.cuda()
    digits = digits = model_class(disease, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTraining Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(300)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    # mirna = mirna.cuda()
    label = label.cuda()
    

    digits = model_class(disease, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Training Epoch[0] Loss:0.38705723732709885 | L1 Loss:0.4750458225607872 | R2:-0.008054308456443615 | ACC: 59.4000%(297/500)
Testing Epoch[0] Loss:0.4496536239981651 | L1 Loss:0.5306484311819076 | R2:-0.09017388554207299 | ACC: 53.0000%(159/300)
Training Epoch[1] Loss:0.3587271347641945 | L1 Loss:0.46102781035006046 | R2:0.06032875772210941 | ACC: 62.4000%(312/500)
Testing Epoch[1] Loss:0.4260165929794312 | L1 Loss:0.5114799350500107 | R2:-0.05148092920864815 | ACC: 53.6667%(161/300)
Training Epoch[2] Loss:0.33272701036185026 | L1 Loss:0.447082344442606 | R2:0.1243728944517026 | ACC: 61.8000%(309/500)
Testing Epoch[2] Loss:0.39884822964668276 | L1 Loss:0.49740455150604246 | R2:0.01124463864517138 | ACC: 55.0000%(165/300)
Training Epoch[3] Loss:0.29524368047714233 | L1 Loss:0.41658322885632515 | R2:0.2225922586131233 | ACC: 66.4000%(332/500)
Testing Epoch[3] Loss:0.38084503412246706 | L1 Loss:0.47566726207733157 | R2:0.03661602313175634 | ACC: 59.6667%(179/300)
Training Epoch[4] Loss:0.2

In [ ]:
print('-----MLP+ MLP 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))


-----MLP+ MLP 300 epochs-----
max test_r2_record  0.31595840723083235
min test_loss_l1_record  0.38764765560626985
min test_loss_record  0.2638873927295208


In [ ]:
print('-----MLP+ MLP 200 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:200]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:200]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:200]))


-----MLP+ MLP 200 epochs-----
max test_r2_record  0.3068995359359115
min test_loss_l1_record  0.3908147007226944
min test_loss_record  0.2664166435599327


In [ ]:
print('-----MLP+ MLP 100 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:100]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:100]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:100]))

-----MLP+ MLP 100 epochs-----
max test_r2_record  0.2979783161699098
min test_loss_l1_record  0.3993300199508667
min test_loss_record  0.2716941736638546


In [ ]:
print('-----MLP+ MLP 50 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:50]))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:50]))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:50]))

-----MLP+ MLP 50 epochs-----
max test_r2_record  0.20738700487910214
min test_loss_l1_record  0.42604627311229704
min test_loss_record  0.30839959979057313
